<a href="https://colab.research.google.com/github/adc257/info4940-sitcom/blob/main/nContext_LLAMA%20with%20some%20results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Big Bang Theory T5 nContext

# Imports:

In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.3 MB/s eta 0:00:00


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
!git clone https://github.com/adc257/info4940-sitcom.git

Cloning into 'info4940-sitcom'...
remote: Enumerating objects: 1104, done.
remote: Counting objects: 100% (1104/1104), done.
remote: Compressing objects: 100% (184/184), done.
remote: Total 1104 (delta 961), reused 1045 (delta 917), pack-reused 0
Receiving objects: 100% (1104/1104), 14.63 MiB | 18.40 MiB/s, done.
Resolving deltas: 100% (961/961), done.


In [3]:
# !pip install -q condacolab
# import condacolab
# condacolab.install()

In [4]:
# !python -m venv 3350
# !source 3350/bin/activate

In [5]:
# !pip install transformers
# !pip install transformers[torch]
# !pip install accelerate -U
# !pip install accelerate>=0.21.0
# !pip install --upgrade pip
# pip uninstall transformers accelerate
# pip install transformers[torch]
# !pip install --upgrade setuptools

# !pip install accelerate

In [6]:
from google.colab import userdata

import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, time
from pathlib import Path

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import os
from sklearn.metrics import f1_score, classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [7]:
# import torch
# import transformers

# from transformers import BertTokenizer, BertForSequenceClassification
# from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, TrainingArguments, Trainer
# from sklearn.model_selection import train_test_split
# from torch.utils.data import DataLoader, TensorDataset
# from tqdm import tqdm
# from transformers import T5Tokenizer, T5ForConditionalGeneration

# Formatting the Dialogue:

In [8]:
def list_files(start_path):
    file_paths = []
    for root, dirs, files in os.walk(start_path):
        for file in files:
            file_paths.append(os.path.join(root, file))

    file_paths.sort()
    return file_paths

In [9]:
def transformContextTarget(context, target):
  separator = "\n"
  quotation_token = '"'

  formatted_input = ""

  current_scene = context[0]['Scene']

  formatted_input += "Use the following as context:" + separator*2
  formatted_input += "The scene is: " + current_scene + separator

  for entry in context:
    if current_scene != entry['Scene']:
      current_scene = entry["Scene"]
      formatted_input += "The scene changes to: " + current_scene + separator

    recipients = entry['Recipients']
    recipients_str = ""
    if len(recipients) == 0:
      recipients_str += "to themselves"
    elif len(recipients) == 1:
      recipients_str += recipients[0]
    else:
      recipients_str += ", ".join(recipients[:-1]) + " and " + recipients[-1]

    formatted_input += f"{entry['Speaker']} says {quotation_token}{entry['Dialogue']}{quotation_token} to {recipients_str}."
    formatted_input += separator


  formatted_input += separator + "-" *20 + separator*2

  formatted_input += "Using the previous dialogues above as context, classify the following dialogue below as humorous or not humorous:" + separator*2


  entry = target
  if entry['Scene'] != current_scene:
    current_scene = entry["Scene"]
    formatted_input += "The scene changes to: "
  else:
    formatted_input += "The Scene is still: "

  formatted_input += current_scene + separator


  recipients = entry['Recipients']
  recipients_str = ""
  if len(recipients) == 0:
    recipients_str += "to themselves"
  elif len(recipients) == 1:
    recipients_str += recipients[0]
  else:
    recipients_str += ", ".join(recipients[:-1]) + " and " + recipients[-1]

  formatted_input += f"{entry['Speaker']} says {quotation_token}{entry['Dialogue']}{quotation_token} to {recipients_str}."
  formatted_input += separator*2 + "Classify: [humorous] or [not humorous]."

  return formatted_input

# context = [data[0], data[1], data[2]]
# target = data[3]
# transformContextTarget(context, target)

In [10]:
def transformContextTarget2(context, target):
  separator = "\n"
  quotation_token = '"'

  formatted_input = ""

  current_scene = context[0]['Scene']

  formatted_input += "User: is the following target_moment from the sitcom, the Big Bang Theory, humorous or not humorous? Only respond 'humorous' or 'not humorous':"
  formatted_input += separator*2
  entry = target
  if entry['Scene'] != current_scene:
    current_scene = entry["Scene"]
    formatted_input += "The scene changes to: "
  else:
    formatted_input += "The Scene is still: "

  formatted_input += current_scene + separator


  recipients = entry['Recipients']
  recipients_str = ""
  if len(recipients) == 0:
    recipients_str += "to themselves"
  elif len(recipients) == 1:
    recipients_str += recipients[0]
  else:
    recipients_str += ", ".join(recipients[:-1]) + " and " + recipients[-1]

  formatted_input += f"{entry['Speaker']} says {quotation_token}{entry['Dialogue']}{quotation_token} to {recipients_str}."

  formatted_input += separator*2 + "Assistant: can you provide some context to help me with this decision?"
  formatted_input += separator*2 + "User: yes, use the following as context:" + separator*2

  # formatted_input += "The scene is: " + current_scene + separator
  if entry['Scene'] != current_scene:
    current_scene = entry["Scene"]
    formatted_input += "The scene changes to: "
  else:
    formatted_input += "The Scene is still: "

  formatted_input += current_scene + separator

  for entry in context:
    if current_scene != entry['Scene']:
      current_scene = entry["Scene"]
      formatted_input += "The scene changes to: " + current_scene + separator

    recipients = entry['Recipients']
    recipients_str = ""
    if len(recipients) == 0:
      recipients_str += "to themselves"
    elif len(recipients) == 1:
      recipients_str += recipients[0]
    else:
      recipients_str += ", ".join(recipients[:-1]) + " and " + recipients[-1]

    formatted_input += f"{entry['Speaker']} says {quotation_token}{entry['Dialogue']}{quotation_token} to {recipients_str}."
    formatted_input += separator


  formatted_input += separator + "-" *20 + separator*2

  formatted_input += "Using the context provided, was the target_moment humorous or not humorous:" + separator*2




  return formatted_input

# context = [data[0], data[1], data[2]]
# target = data[3]
# transformContextTarget(context, target)

In [11]:
def transformContextTarget3(context, target):
  messages = []

  separator = "\n"
  quotation_token = '"'

  # formatted_input = ""

  systemPrmpt = {
      'role': 'system',
      'content': "You are humor classification model that takes a given line (the target) along with several precending lines (the context), and determines whether the target line was humorous (return '1') or not humorous (return '0'). RESTRICTION: ONLY RESPOND 1 OR 0."
  }
  # append systemPrmpt
  messages.append(systemPrmpt)

  systemPrmpt2 = {
      'role': 'system',
      'content': ""
  }
  userPrmpt = {
      'role': 'user',
      'content': ""
  }
  userPrmpt2 = {
      'role': 'user',
      'content': ""
  }
  asstPrmpt = {
      'role': 'assistant',
      'content': ""
  }

  current_scene = context[0]['Scene']

  userPrmpt['content'] += "Is the following target_moment from the sitcom, the Big Bang Theory, humorous or not humorous? Only respond '1' for humorous or '0' for not humorous:"
  userPrmpt['content'] += separator*2
  entry = target
  if entry['Scene'] != current_scene:
    current_scene = entry["Scene"]
    userPrmpt['content'] += "The scene changes to: "
  else:
    userPrmpt['content'] += "The Scene is still: "

  userPrmpt['content'] += current_scene + separator


  recipients = entry['Recipients']
  recipients_str = ""
  if len(recipients) == 0:
    recipients_str += "to themselves"
  elif len(recipients) == 1:
    recipients_str += recipients[0]
  else:
    recipients_str += ", ".join(recipients[:-1]) + " and " + recipients[-1]

  userPrmpt['content'] += f"{entry['Speaker']} says {quotation_token}{entry['Dialogue']}{quotation_token} to {recipients_str}."

  # append userPrmpt
  messages.append(userPrmpt)

  asstPrmpt['content'] += "Can you provide some context to help me with this decision?"
  # append asstPrmpt
  messages.append(asstPrmpt)

  userPrmpt2['content'] += "Yes, use the following as context:" + separator*2

  if entry['Scene'] != current_scene:
    current_scene = entry["Scene"]
    userPrmpt2['content'] += "The scene changes to: "
  else:
    userPrmpt2['content'] += "The Scene is still: "

  userPrmpt2['content'] += current_scene + separator

  for entry in context:
    if current_scene != entry['Scene']:
      current_scene = entry["Scene"]
      userPrmpt2['content'] += "The scene changes to: " + current_scene + separator

    recipients = entry['Recipients']
    recipients_str = ""
    if len(recipients) == 0:
      recipients_str += "to themselves"
    elif len(recipients) == 1:
      recipients_str += recipients[0]
    else:
      recipients_str += ", ".join(recipients[:-1]) + " and " + recipients[-1]

    userPrmpt2['content'] += f"{entry['Speaker']} says {quotation_token}{entry['Dialogue']}{quotation_token} to {recipients_str}."
    userPrmpt2['content'] += separator

  # append userPrmpt2
  messages.append(userPrmpt2)

  # userPrmpt2['content'] += separator + "-" *20 + separator*2

  systemPrmpt2['content'] += "Using the context provided, return '1' if the target_moment was humorous or '0' if the target_moment was not humorous?"

  # append systemPrmpt2
  messages.append(systemPrmpt2)

  # messages = [systemPrmpt, userPrmpt, asstPrmpt, ]

  return messages

# context = [data[0], data[1], data[2]]
# target = data[3]
# transformContextTarget(context, target)

In [12]:
filePath = "/content/info4940-sitcom/cleaned-data/S1"
nSentenceContext = 3

episodePath = list_files(filePath)[0]

with open(episodePath, 'r') as file:
  data = json.load(file)
  data = [info for (_, info) in data.items()]

print(f"Example raw entry:")
display(data[3])

print("\n\nExample formatted entry (3context):\n")
print(transformContextTarget3(data[:3], data[3]))

print(f'Actual: {("isHumor" in data[3])}')

Example raw entry:


{'EpisodeID': 'The Big Bang_S0101',
 'Scene': 'A corridor at a sperm bank.',
 'Recipients': ['Sheldon', 'Receptionist'],
 'Speaker': 'Leonard',
 'Dialogue': 'Excuse me.',
 'Dialogue Start Time': '00:00:23:140000',
 'Dialogue End Time': '00:00:25:060000'}



Example formatted entry (3context):

[{'role': 'system', 'content': "You are humor classification model that takes a given line (the target) along with several precending lines (the context), and determines whether the target line was humorous (return '1') or not humorous (return '0'). RESTRICTION: ONLY RESPOND 1 OR 0."}, {'role': 'user', 'content': 'Is the following target_moment from the sitcom, the Big Bang Theory, humorous or not humorous? Only respond \'1\' for humorous or \'0\' for not humorous:\n\nThe Scene is still: A corridor at a sperm bank.\nLeonard says "Excuse me." to Sheldon and Receptionist.'}, {'role': 'assistant', 'content': 'Can you provide some context to help me with this decision?'}, {'role': 'user', 'content': 'Yes, use the following as context:\n\nThe Scene is still: A corridor at a sperm bank.\nSheldon says "So if a photon is directed through a plane with two slits in it and either slit is observed, it will not go through both slits. If it\'s unobserved, it wi

In [13]:
for i in transformContextTarget3(data[:3], data[3]):
  print("role: ", i['role'])
  print(i['content'])

role:  system
You are humor classification model that takes a given line (the target) along with several precending lines (the context), and determines whether the target line was humorous (return '1') or not humorous (return '0'). RESTRICTION: ONLY RESPOND 1 OR 0.
role:  user
Is the following target_moment from the sitcom, the Big Bang Theory, humorous or not humorous? Only respond '1' for humorous or '0' for not humorous:

The Scene is still: A corridor at a sperm bank.
Leonard says "Excuse me." to Sheldon and Receptionist.
role:  assistant
Can you provide some context to help me with this decision?
role:  user
Yes, use the following as context:

The Scene is still: A corridor at a sperm bank.
Sheldon says "So if a photon is directed through a plane with two slits in it and either slit is observed, it will not go through both slits. If it's unobserved, it will. However, if it's observed after it's left the plane but before it hits its target, it won't have gone through both slits." t

# Train Test Split:

In [14]:
def generateDialogueWithContext(filePath, nSentenceContext):
  formatted_inputs = []
  labels = []

  for episodePath in list_files(filePath):

    with open(episodePath, 'r') as file:
      data = json.load(file)
      data = [info for (_, info) in data.items()]


      i = nSentenceContext
      while i < len(data):
        context = [data[j] for j in range(i-nSentenceContext, i)]
        target = data[i]

        formatted_inputs.append(transformContextTarget3(context, target))

        if "isHumor" in target:
          labels.append(1)
        else:
          labels.append(0)

        i += 1
  return formatted_inputs, labels

In [15]:
nContext = 5
dialogue_list1, label_list1 = generateDialogueWithContext('/content/info4940-sitcom/cleaned-data/S1', nContext)
dialogue_list2, label_list2 = generateDialogueWithContext('/content/info4940-sitcom/cleaned-data/S2', nContext)
dialogue_list3, label_list3 = generateDialogueWithContext('/content/info4940-sitcom/cleaned-data/S3', nContext)
dialogue_list4, label_list4 = generateDialogueWithContext('/content/info4940-sitcom/cleaned-data/S4', nContext)
dialogue_list5, label_list5 = generateDialogueWithContext('/content/info4940-sitcom/cleaned-data/S5', nContext)

In [16]:
train_texts = dialogue_list1 + dialogue_list2 + dialogue_list3 + dialogue_list4
test_texts = dialogue_list5
train_labels = label_list1 + label_list2 + label_list3 + label_list4
test_labels = label_list5

In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id, token=userdata.get('HF_TOKEN'))
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    token=userdata.get('HF_TOKEN')
)


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

In [18]:
# messages = [
#     {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
#     {"role": "user", "content": "Who are you?"},
# ]

messages = test_texts[0]

input_ids = tokenizer.apply_chat_template(
    messages[:-1],
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=50,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


1


In [19]:
messages[:-1]

[{'role': 'system',
  'content': "You are humor classification model that takes a given line (the target) along with several precending lines (the context), and determines whether the target line was humorous (return '1') or not humorous (return '0'). RESTRICTION: ONLY RESPOND 1 OR 0."},
 {'role': 'user',
  'content': 'Is the following target_moment from the sitcom, the Big Bang Theory, humorous or not humorous? Only respond \'1\' for humorous or \'0\' for not humorous:\n\nThe Scene is still: The Cafeteria\nSheldon says "It\'s not what it looks like." to Leonard.'},
 {'role': 'assistant',
  'content': 'Can you provide some context to help me with this decision?'},
 {'role': 'user',
  'content': 'Yes, use the following as context:\n\nThe Scene is still: The Cafeteria\nPriya says "My brother-- he\'s got a big crush on Bernadette." to Leonard.\nLeonard says "You\'re moving back to India?" to Priya.\nLeonard says "What\'s going on?" to Raj, Penny, Raj and Howard.\nPenny says "It\'s-It\'s n

In [ ]:

print("""Based on the context, I would classify the target moment as humorous. The phrase "It's not what it looks like" is a common idiomatic expression, but in this scene, it's being used in a way that's reminiscent of a soap opera or a melodrama, which adds a layer of humor to the situation. The fact that Sheldon is asking "What does it look like?" also implies that he's being obtuse and unaware of the obvious, which is a characteristic that's often played for comedic effect in the show.""")


Based on the context, I would classify the target moment as humorous. The phrase "It's not what it looks like" is a common idiomatic expression, but in this scene, it's being used in a way that's reminiscent of a soap opera or a melodrama, which adds a layer of humor to the situation. The fact that Sheldon is asking "What does it look like?" also implies that he's being obtuse and unaware of the obvious, which is a characteristic that's often played for comedic effect in the show.


In [27]:
success = 0
fail = 0
total = 0

for i in range(len(test_texts)):
  input = test_texts[i]
  expOutput = test_labels[i]
  # if expOutput == 1:
  #   expOutput = "humorous"
  # else:
  #   expOutput = "not humorous"

  input_ids = tokenizer.apply_chat_template(
      input[:-1],
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=50
      ,
      eos_token_id=terminators,
      # do_sample=True,
      # temperature=0.6,
      # top_p=0.9,
      num_return_sequences=1,
      temperature=0.5,
      do_sample=True
  )
  response = outputs[0][input_ids.shape[-1]:]
  response = tokenizer.decode(response, skip_special_tokens=True)

  if str(response) != str(expOutput):
    fail += 1

    print("\nFail:")
    print(f"Input: {input}")
    print(f"Expected: {expOutput}, {type(expOutput)}, model: {response}, {type(response)}")
    print("\n")

  else:
    success += 1
    print("\nSuccess:")
    # print(f"Input: {input}")
    # print(f"Expected: {expOutput}, {type(expOutput)}, model: {response}, {type(response)}")
    # print("\n")

  total += 1

  print(f"Test: {i} of {len(test_texts)}\n\n")
  print(f"Success rate: {success/total:.2%}")



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Success:
Test: 0 of 5131


Success rate: 100.00%


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Success:
Test: 1 of 5131


Success rate: 100.00%


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Fail:
Input: [{'role': 'system', 'content': "You are humor classification model that takes a given line (the target) along with several precending lines (the context), and determines whether the target line was humorous (return '1') or not humorous (return '0'). RESTRICTION: ONLY RESPOND 1 OR 0."}, {'role': 'user', 'content': 'Is the following target_moment from the sitcom, the Big Bang Theory, humorous or not humorous? Only respond \'1\' for humorous or \'0\' for not humorous:\n\nThe scene changes to: The Cheesecake Factory.\nHoward says "Thanks for dinner, buddy." to Raj, Sheldon, Leonard and Penny.'}, {'role': 'assistant', 'content': 'Can you provide some context to help me with this decision?'}, {'role': 'user', 'content': 'Yes, use the following as context:\n\nThe Scene is still: The Cheesecake Factory.\nThe scene changes to: Penny’s apartment.\nRaj says "Look, Penny, you\'re great, but I had a long talk with my parents, and they said if I date an Indian girl, I get a Maserati." 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Fail:
Input: [{'role': 'system', 'content': "You are humor classification model that takes a given line (the target) along with several precending lines (the context), and determines whether the target line was humorous (return '1') or not humorous (return '0'). RESTRICTION: ONLY RESPOND 1 OR 0."}, {'role': 'user', 'content': 'Is the following target_moment from the sitcom, the Big Bang Theory, humorous or not humorous? Only respond \'1\' for humorous or \'0\' for not humorous:\n\nThe Scene is still: The Cheesecake Factory.\nLeonard says "Yeah, real big of you." to Raj, Howard, Sheldon and Penny.'}, {'role': 'assistant', 'content': 'Can you provide some context to help me with this decision?'}, {'role': 'user', 'content': 'Yes, use the following as context:\n\nThe Scene is still: The Cheesecake Factory.\nRaj says "Mmm. Oh, cheesecake, you\'re just as good as a woman, even though I can\'t have sex with you." to Howard, Sheldon and Leonard.\nHoward says "Try throwing it in the microwave

KeyboardInterrupt: 

In [28]:
print(f"Total: {total}, Success rate: {success/total:.2%}")

Total: 881, Success rate: 53.58%


In [26]:
messages = [
#     {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "What do you know about the sitcom the Big Bang Theory?"},
]

# messages = test_texts[0]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A popular and iconic sitcom! The Big Bang Theory is a American sitcom that originally aired from 2007 to 2019. Created by Chuck Lorre and Bill Prady, the show is a comedy about a group of socially awkward and intellectually brilliant scientists and their neighbor, a waitress and aspiring actress.

The show revolves around the lives of five main characters:

1. Sheldon Cooper (played by Jim Parsons), a theoretical physicist with Asperger's syndrome and a strict adherence to routine.
2. Leonard Hofstadter (played by Johnny Galecki), an experimental physicist and Sheldon's roommate and best friend.
3. Howard Wolowitz (played by Simon Helberg), an aerospace engineer and fellow physicist who lives across the hall from Leonard and Sheldon.
4. Rajesh "Raj" Koothrappali (played by Kunal Nayyar), an astrophysicist who is unable to speak in the presence of women unless he drinks alcohol.
5. Penny (played by Kaley Cuoco), a waitress and aspiring actress who lives across the hall from Leonard and 

In [ ]:
# messages = [
#     {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
#     {"role": "user", "content": "Who are you?"},
# ]



input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Arrrr, me hearty! Me name be Captain Chat, the scurviest chatbot to ever sail the Seven Seas o' Conversation! Me be a swashbucklin' automaton, here to chat with ye about all manner o' things, from the treasure o' knowledge to the booty o' banter! So hoist the colors, me hearty, and let's set sail fer a chat like no other!


In [ ]:
tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
    tokenize=False
)

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a pirate chatbot who always responds in pirate speak!<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWho are you?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'

# T5:

In [ ]:
# tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xxl")
# model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xxl", device_map="auto")

In [ ]:
# Save the tokenizer and model
# tokenizer.save_pretrained("flan_t5_xxl_tokenizer")
# model.save_pretrained("flan_t5_xxl_model")

In [ ]:
## Check whether it saved properly

# directory = "flan_t5_xxl_tokenizer"

# if os.path.exists(directory):
#     print(f"Files in '{directory}': {os.listdir(directory)}")
# else:
#     print(f"The directory '{directory}' does not exist.")

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small", device_map="auto")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
input_text1 = 'Use the following as context:\n\nThe scene is: A corridor at a sperm bank.\nSheldon says "So if a photon is directed through a plane with two slits in it and either slit is observed, it will not go through both slits. If it\'s unobserved, it will. However, if it\'s observed after it\'s left the plane but before it hits its target, it won\'t have gone through both slits." to Leonard.\nLeonard says "Agreed. What\'s your point?" to Sheldon.\nSheldon says "There\'s no point, I just think it\'s a good idea for a T-shirt." to Leonard.\n\n--------------------\n\n Using the previous dialogues above as context, classify the following dialogue below as humorous:\n\nThe Scene is still: A corridor at a sperm bank.\nLeonard says "Excuse me." to Sheldon and Receptionist. Is this funny?'
input_ids1 = tokenizer(input_text1, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids1, max_new_tokens=5)
tokenizer.decode(outputs[0], skip_special_tokens=True)

'Yes -----------'

In [ ]:
def classify_text(generated_text):
    if "not humorous" in generated_text.lower():
        return 0  # Not Humorous
    else:
        return 1  # Humorous

predictions = []

for text in train_texts:
    input_ids = tokenizer(text, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(input_ids, max_new_tokens=5)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Classify the generated text
    prediction = classify_text(generated_text)
    predictions.append(prediction)


In [ ]:
predictions.count(0), len(predictions)

(105, 105)

In [ ]:
input_text1 = "question: What does increased oxygen concentrations in the patient’s lungs displace? context: Hyperbaric (high-pressure) medicine uses special oxygen chambers to increase the partial pressure of O 2 around the patient and, when needed, the medical staff. Carbon monoxide poisoning, gas gangrene, and decompression sickness (the ’bends’) are sometimes treated using these devices. Increased O 2 concentration in the lungs helps to displace carbon monoxide from the heme group of hemoglobin. Oxygen gas is poisonous to the anaerobic bacteria that cause gas gangrene, so increasing its partial pressure helps kill them. Decompression sickness occurs in divers who decompress too quickly after a dive, resulting in bubbles of inert gas, mostly nitrogen and helium, forming in their blood. Increasing the pressure of O 2 as soon as possible is part of the treatment."
input_ids1 = tokenizer(input_text1, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids1, max_new_tokens=5)
tokenizer.decode(outputs[0], skip_special_tokens=True)

'carbon monoxide'

In [ ]:
# input_text1 = "question: Is the following dialogue funny? \"That\'s probably enough about us. So, tell us about you.\" context: 'That has negative social implications.', 'So... Klingon Boggle?', 'Yeah. It\'s like regular Boggle, but... in Klingon.'"

# # context: Hyperbaric (high-pressure) medicine uses special oxygen chambers to increase the partial pressure of O 2 around the patient and, when needed, the medical staff. Carbon monoxide poisoning, gas gangrene, and decompression sickness (the ’bends’) are sometimes treated using these devices. Increased O 2 concentration in the lungs helps to displace carbon monoxide from the heme group of hemoglobin. Oxygen gas is poisonous to the anaerobic bacteria that cause gas gangrene, so increasing its partial pressure helps kill them. Decompression sickness occurs in divers who decompress too quickly after a dive, resulting in bubbles of inert gas, mostly nitrogen and helium, forming in their blood. Increasing the pressure of O 2 as soon as possible is part of the treatment."
# input_ids1 = tokenizer(input_text1, return_tensors="pt").input_ids.to("cuda")

# outputs = model.generate(input_ids1, max_new_tokens=5)
# tokenizer.decode(outputs[0], skip_special_tokens=True)